### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/07/16 23:50:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/07/16 23:50:18 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


### Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Drug Exposure

In [5]:
## Quantidade de exposicao a droga muito baixo pois não existe um start date
df_drug_exposure = (df_input
                  .select('person_id', 'dt_inic_tr', 'visit_occurrence_id')
                  .withColumn('drug_concept_id', F.when(F.col('dt_inic_tr').isNotNull(), 21090430))
                  .withColumn('drug_exposure_start_date', F.when(F.col('dt_inic_tr').isNotNull(), F.to_date(F.col('dt_inic_tr'), 'yyyy-MM-dd')).otherwise(None))
                  .withColumn('drug_exposure_end_date', F.when(F.col('dt_inic_tr').isNotNull(), F.date_add('drug_exposure_start_date', 180)))
                  .withColumn('drug_type_concept_id', F.lit(32879))
                  .withColumn('quantity', F.lit(None).cast('float'))
                  .withColumn('dose_unit_source_value', F.lit(None).cast('string'))
                    .withColumn('drug_exposure_end_datetime', F.lit(None))
                    .withColumn('drug_exposure_start_datetime', F.lit(None))
                    .withColumn('verbatim_end_date', F.lit(None))
                    .withColumn('stop_reason', F.lit(None))
                    .withColumn('refills', F.lit(None))
                    .withColumn('days_supply', F.lit(None))
                    .withColumn('sig', F.lit(None))
                    .withColumn('route_concept_id', F.lit(None))
                    .withColumn('lot_number', F.lit(None))
                    .withColumn('provider_id', F.lit(None))
                    .withColumn('visit_occurrence_id', F.col('visit_occurrence_id'))
                    .withColumn('visit_detail_id', F.lit(None))
                    .withColumn('drug_source_value', F.lit(None))
                    .withColumn('drug_source_concept_id', F.lit(None))
                    .withColumn('route_source_value', F.lit(None))
                    .withColumn('drug_exposure_id', F.row_number().over(Window.orderBy('person_id')))
                    .filter(F.col('drug_exposure_start_date').isNotNull())
           ).select(
                    F.col('drug_exposure_id').cast('integer'),
                    F.col('person_id').cast('integer'),
                    F.col('drug_concept_id').cast('integer'),
                    F.col('drug_exposure_start_date').cast('date'),
                    F.col('drug_exposure_start_datetime').cast('timestamp'),
                    F.col('drug_exposure_end_date').cast('date'),
                    F.col('drug_exposure_end_datetime').cast('timestamp'),
                    F.col('verbatim_end_date').cast('date'),
                    F.col('drug_type_concept_id').cast('integer'),
                    F.col('stop_reason').cast('string'),
                    F.col('refills').cast('integer'),
                    F.col('quantity').cast('float'),
                    F.col('days_supply').cast('integer'),
                    F.col('sig').cast('string'),
                    F.col('route_concept_id').cast('integer'),
                    F.col('lot_number').cast('string'),
                    F.col('provider_id').cast('integer'),
                    F.col('visit_occurrence_id').cast('integer'),
                    F.col('visit_detail_id').cast('integer'),
                    F.col('drug_source_value').cast('string'),
                    F.col('drug_source_concept_id').cast('integer'),
                    F.col('route_source_value').cast('string'),
                    F.col('dose_unit_source_value').cast('string')
                )

df_drug_exposure.count()

17058

## Salvando no postgresql

In [6]:
write_to_postgresl(df_drug_exposure, tb_name='tuberculose.drug_exposure', write_mode='append')

25/07/16 23:56:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:56:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:56:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:56:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:56:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'Total time: 0:01:01.307374 and - Total rows: 17058 - Total columns: 23'